In [ ]:
import pandas as pd
import numpy as np
import pickle
from tqdm import tqdm

In [ ]:
# duplicate answer question data remove
df_eedi = pd.read_csv('./data/eedi_a/train_task_1_2.csv')
df_eedi

filtered_df = df_eedi.groupby('QuestionId').filter(lambda x: x[(x.IsCorrect == 1)]['CorrectAnswer'].nunique() <= 1 and x[(x.IsCorrect == 0)]['CorrectAnswer'].nunique() <= 1)
null_list = df_eedi[~df_eedi.index.isin(filtered_df.index)]['QuestionId'].unique().tolist()
# null_list

In [ ]:
len(null_list)

In [ ]:
# baseline id_dic_construction
def id_dic_construction(x):
    corresponding_dic = {}
    for dic_index in range(len(x)):
        corresponding_dic[x[dic_index]] = dic_index
    return corresponding_dic

In [ ]:
# baseline id_dic

# Extract question_info 
question_info = pd.read_csv('./data/eedi_a/question_metadata_task_1_2.csv')
print(question_info.columns)
question_info = question_info.drop_duplicates()
for q in tqdm(null_list):
    question_info = question_info.drop(question_info[question_info['QuestionId'] == q].index)
question_info = question_info.values

# Extract question_id
question_id = np.unique(question_info[:, 0])
question_dic = id_dic_construction(question_id)

# Extract skill_id
for i, kc in enumerate(tqdm(question_info)):
    question_info[i, 1] = list(map(int, kc[1][1:-1].split(',')))
skill = []
for i in tqdm(range(len(question_info))):
    skill += question_info[i, 1]
skill = np.unique(np.array(skill).astype('int64'))
skill_dic = id_dic_construction(skill)

# question, skill renaming with dictionary
for i in tqdm(range(len(question_info))):
    question_info[i, 0] = question_dic[question_info[i, 0]]
    for n in range(len(question_info[i, 1])):
        question_info[i, 1][n] = skill_dic[question_info[i, 1][n]]
question_info

In [ ]:
# Make baseline q_matrix
tmp_question_skill_cell = []
for i in tqdm(range(len(question_info))):
    q_id = question_info[i, 0]
    skill = question_info[i, 1]
    for kc in np.array(skill).astype('int64'):
        tmp_question_skill_cell.append([q_id, kc])
tmp_question_skill_cell = np.array(tmp_question_skill_cell)
exercise_num = len(question_dic)
skill_num = len(skill_dic)
tmp_exercise_id = tmp_question_skill_cell[:, 0]
tmp_skill_id = tmp_question_skill_cell[:, 1]
baseline_q_matrix = np.zeros([exercise_num, skill_num])
for i in tqdm(range(len(tmp_question_skill_cell))):
    baseline_q_matrix[int(tmp_exercise_id[i]), int(tmp_skill_id[i])] = 1
baseline_q_matrix

In [ ]:
with open('./data/eedi_a/q_matrix_eedi_a.pkl', 'wb') as f:
    pickle.dump(baseline_q_matrix, f)

In [ ]:
# Extract student id
# student_info = pd.read_csv('./data/eedi_a/student_metadata_task_1_2.csv')
# student_info = student_info.drop_duplicates()
# student_info = student_info.values

# student_id = np.unique(student_info[:, 0])
# student_dic = id_dic_construction(student_id)

In [ ]:
answer_info = pd.read_csv('./data/eedi_a/answer_metadata_task_1_2.csv')
answer_info

### Baseline Preprocess

In [ ]:
df_eedi = pd.read_csv('./data/eedi_a/train_task_1_2.csv')
df_eedi

In [ ]:
# duplicate answer question data remove
filtered_df_eedi = df_eedi.groupby('QuestionId').filter(lambda x: x[(x.IsCorrect == 1)]['CorrectAnswer'].nunique() <= 1 and x[(x.IsCorrect == 0)]['CorrectAnswer'].nunique() <= 1)
df_eedi = filtered_df_eedi
df_eedi

In [ ]:
# with open('./data/eedi_a/eedi_a_noise_remove_data.pkl', 'wb') as f:
#     pickle.dump(df_eedi, f)

In [ ]:
# df_eedi = pd.read_pickle('./data/eedi_a/eedi_a_noise_remove_data.pkl')
# df_eedi

In [ ]:
# dataframe question renaming
df_eedi.iloc[:, 0] = df_eedi.iloc[:, 0].map(question_dic)

In [ ]:
question_df = pd.DataFrame([(question[0], question[1]) for question in question_info], columns=["QuestionId", "SubjectId"])
question_df

In [ ]:
# merge question df
df_eedi = pd.merge(df_eedi, question_df, how='inner', on='QuestionId')
df_eedi

In [ ]:
df_eedi = df_eedi[['UserId', 'QuestionId', 'AnswerValue', 'AnswerId', 'CorrectAnswer', 'SubjectId', 'IsCorrect']]
df_eedi.columns = ['UserId', 'QuestionId', 'AnswerValue', 'AnswerId', 'CorrectAnswer', 'tag', 'IsCorrect']
df_eedi

In [ ]:
df_eedi_values = df_eedi.values

In [ ]:
# Problem solving rate of each student
from collections import defaultdict

problem_solving_rate = defaultdict(int)
grouped = df_eedi.groupby(df_eedi.iloc[:, 0])
for student, group in tqdm(grouped):
    unique_questions = group.iloc[:, 1].unique()
    problem_solving_rate[student] = len(unique_questions) / len(question_dic)
dict(problem_solving_rate)

In [ ]:
# Percentage of total correct answers for each student
correct_rate = defaultdict(int)
grouped = df_eedi.groupby(df_eedi.iloc[:, 0])
for student, group in tqdm(grouped):
    if len(group.iloc[:, 6]) == 0:
        correct_rate[student] = 0
    else:
        correct_rate[student] = sum(group.iloc[:, 6])/len(group.iloc[:, 6])
correct_rate

In [ ]:
# A dicttionary with a list of students who solved each problem
problem_student = defaultdict(int)
grouped = df_eedi.groupby(df_eedi.iloc[:, 1])
for question, group in tqdm(grouped):
    unique_students = group.iloc[:, 0].unique()
    problem_student[question] = unique_students
problem_student

In [ ]:
# Pick leading group
leading_group = {}
problem_solving_rate_upper_bound = 0.4 # problem solving upper bound
correct_rate_upper_bound = 0.4 # correct rate upper bound
for key in tqdm(problem_student):
    tmp = np.empty((0, 3), float)
    for student in problem_student[key]:
        tmp = np.append(tmp, np.array([[student, problem_solving_rate[student], correct_rate[student]]]), axis=0)
    pro_solv_rate_sorted_idx = np.argsort(tmp[:, 1])[::-1]
    tmp = tmp[pro_solv_rate_sorted_idx]
    pro_solv_rate_up_bound = int(np.round(len(tmp[:, 1])*problem_solving_rate_upper_bound))
    tmp = tmp[:pro_solv_rate_up_bound+1]
    cor_rate_sorted_idx = np.argsort(tmp[:, 2])[::-1]
    tmp = tmp[cor_rate_sorted_idx]
    cor_rate_up_bound = int(np.round(len(tmp[:, 2])*correct_rate_upper_bound))
    tmp_leading_group = tmp[:cor_rate_up_bound+1]
    leading_group[key] = tmp_leading_group[:, 0].astype(int)
# leading_group

In [ ]:
# Option weight extraction based on the option selection rate of the leading group

opt_dic = {}
for key, group in tqdm(df_eedi.groupby('QuestionId')):
    tmp_opt_list = []
    tmp_opt_dic = {}
    for i in leading_group[key]:
        filtered_df = group[group['UserId'] == i]
        tmp_opt_list += list(filtered_df['AnswerValue'])
        correct_answer = list(filtered_df['CorrectAnswer'])[0]
    tmp_opt_dic[1] = tmp_opt_list.count(1) / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic[2] = tmp_opt_list.count(2) / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic[3] = tmp_opt_list.count(3) / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic[4] = tmp_opt_list.count(4) / len(tmp_opt_list) if len(tmp_opt_list) != 0 else 0
    tmp_opt_dic[correct_answer] = 2.0
    sorted_dic = dict(sorted(tmp_opt_dic.items(), key=lambda x: x[1]))
    sorted_keys = list(sorted_dic.keys())
    tmp_opt_dic[sorted_keys[0]] = 1
    tmp_opt_dic[sorted_keys[1]] = 2
    tmp_opt_dic[sorted_keys[2]] = 3
    tmp_opt_dic[sorted_keys[3]] = 4
    opt_dic[key] = tmp_opt_dic


In [ ]:
df_eedi

In [ ]:
df_eedi_values

In [ ]:
# Extract Option Weight
option_list = []
for i in tqdm(range(len(df_eedi_values))):
    option_list.append(opt_dic[df_eedi_values[i, 1]][df_eedi_values[i, 2]])
df_eedi['OptionWeight'] = option_list
df_eedi

In [ ]:
# extract if it is wrong, but the option weight is 4 ==> nothing should come out, it is normal
df_eedi[(df_eedi['IsCorrect']==0)&(df_eedi['OptionWeight']==4)]


In [ ]:
# Save pickle file
# with open(f'./data/eedi/method2/base_eedi_a_option_{problem_solving_rate_upper_bound}_{correct_rate_upper_bound}.pkl', 'wb') as f:
#     pickle.dump(df_eedi, f)
with open(f'./data/eedi_a/method_2/eedi_a.pkl', 'wb') as f:
    pickle.dump(df_eedi, f)

### Timestamp Add

In [ ]:
tmp_answer_info = answer_info[['AnswerId', 'DateAnswered']]
df_eedi = pd.merge(df_eedi, tmp_answer_info, how='inner', on='AnswerId')
df_eedi

In [ ]:
import datetime
import time
# datetime to timestamp 
eedi_time = list(df_eedi['DateAnswered'])
for i, timestr in enumerate(tqdm(eedi_time)):    
    eedi_time[i] = time.mktime(datetime.datetime.strptime(str(timestr[:-4]), "%Y-%m-%d %H:%M:%S").timetuple())
eedi_time


In [ ]:
# save timestamp

df_eedi['DateAnswered'] = eedi_time
df_eedi = df_eedi.sort_values(by=['DateAnswered'])
df_eedi.reset_index(drop = True, inplace = True)
df_eedi

In [ ]:
with open(f'../data/eedi_a/method_2/eedi_a.pkl', 'wb') as f:
    pickle.dump(df_eedi, f)